In [1]:
import numpy as np
from keras.datasets import imdb
from matplotlib import pyplot
import os

os.environ['KMP_DUPLICATE_LIB_OK']='True'

NUM_WORDS = 5000

# save np.load
np_load_old = np.load

# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

# call load_data with allow_pickle implicitly set to true
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words = NUM_WORDS)

np.load = np_load_old


Using TensorFlow backend.


In [2]:
print(X_train[0])
print(y_train[0])
print(y_test[0])
print(X_test[0])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 2, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 2, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 2, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 2, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 2, 19, 178, 32]
1
0
[1, 591, 202, 14, 31, 6, 717, 10, 10, 2, 2, 5, 4, 360, 7, 4, 177, 2, 39

In [3]:
print(len(np.unique(np.hstack(X_train))))

4998


In [4]:
# Truncating and Padding the dataset to a length of 500 words for each observation.
from keras.preprocessing import sequence
X_train = sequence.pad_sequences(X_train, maxlen=500)
X_test = sequence.pad_sequences(X_test, maxlen=500)

In [5]:
from keras.layers import Embedding
Embedding(NUM_WORDS, 32, input_length = 500)

W0903 16:00:27.577137 140735151964160 deprecation_wrapper.py:119] From /Users/flock/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



In [7]:
from keras.models import Sequential
from keras.layers import Flatten, LSTM
from keras.layers import Dense
model = Sequential()
model.add(Embedding(NUM_WORDS, 32, input_length = 500))
# model.add(Flatten())
# model.add(Dense(250, activation='relu'))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

W0903 16:01:53.019290 140735151964160 deprecation_wrapper.py:119] From /Users/flock/anaconda3/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0903 16:01:53.111073 140735151964160 deprecation_wrapper.py:119] From /Users/flock/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3376: The name tf.log is deprecated. Please use tf.math.log instead.

W0903 16:01:53.118671 140735151964160 deprecation.py:323] From /Users/flock/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
None


In [8]:
#fitting the model
model.fit(X_train, y_train, epochs=2, batch_size=128, validation_data=(X_test, y_test), verbose=1)


W0903 16:01:57.632255 140735151964160 deprecation_wrapper.py:119] From /Users/flock/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 773s 31ms/step - loss: 0.7080 - acc: 0.6346 - val_loss: 0.5248 - val_acc: 0.7844
Epoch 2/2
25000/25000 [==============================] - 821s 33ms/step - loss: 0.4309 - acc: 0.8177 - val_loss: 0.3370 - val_acc: 0.8588


In [9]:
#final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print(scores[1]*100)

85.88
